## Importing Packages

In [30]:
seal = ['adult_males', 'subadult males', 'adult females', 'juveniles', 'pups']
rgb = [(243,8,5), (244,8,242), (87,46,10), (25,56,176), (38,174,21)]  
cls = [0, 1, 2, 3, 4]
hex = ['#f30805', '#f408f2', '#572e0a', '#1938b0', '#26ae15']

In [31]:
seal2rgb = dict(zip(seal,rgb))
cls2rgb = dict(zip(cls, rgb))
cls2hex = dict(zip(cls, hex))
seal2cls = dict(zip(seal,cls))
cls2seal = dict(zip(cls,seal))
rgb2cls = dict(zip(rgb, cls))

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, Input, concatenate
from keras.models import Model
from keras import backend as K
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from PIL import  Image
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt

n_classes= 5
batch_size= 4
epochs= 100
image_size= 360
img_sz = (512,512)
model_name= 'cnn_regression_moi'

### Loading file names

In [11]:
trn_ids = pd.read_csv('../IDsTrainingSet.txt')
tst_ids = pd.read_csv('../IDsforBlending.txt')

In [12]:
trn_ids.shape, tst_ids.shape

((0, 657), (0, 250))

In [13]:
trn_ids = np.array(trn_ids.columns)
tst_ids = np.array(tst_ids.columns)

In [14]:
len(trn_ids), len(tst_ids)

(657, 250)

In [15]:
fnames = ['../NOA_sealions/Train/'+str(int(i))+'.jpg' for i in trn_ids]
tnames = ['../NOA_sealions/Train/'+str(int(i))+'.jpg' for i in tst_ids]

In [16]:
tnames[0:3]

['../NOA_sealions/Train/513.jpg',
 '../NOA_sealions/Train/2.jpg',
 '../NOA_sealions/Train/516.jpg']

In [17]:
def open_image(fn): return np.array(Image.open(fn).resize(img_sz, Image.NEAREST))
imgs = np.stack([open_image(fn) for fn in fnames])

In [38]:
cord = pd.read_csv('../coords/coords.csv')

In [19]:
df= pd.read_csv('../NOA_sealions/Train/train.csv')
n_train_images= len(fnames)
y_list=[]
for i in range(0,n_train_images): 
    row= df.ix[i] 
    y_row= np.zeros((5))
    y_row[0]= row['adult_males']
    y_row[1]= row['subadult_males']
    y_row[2]= row['adult_females']
    y_row[3]= row['juveniles']
    y_row[4]= row['pups']
    y_list.append(y_row)

x_train= np.asarray(imgs)
y_train= np.asarray(y_list)

print('x_train.shape', x_train.shape)
print('y_train.shape', y_train.shape)


x_train.shape (657, 512, 512, 3)
y_train.shape (657, 5)


/home/syed_dani8/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


In [20]:
input_shape = (image_size, image_size, 3)
input_shape

(360, 360, 3)

In [24]:
input_shape = (image_size, image_size, 3)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(n_classes, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())

print (model.summary())
#sys.exit(0) #

model.compile(loss=keras.losses.mean_squared_error,
        optimizer= keras.optimizers.adam(lr=0.01))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 360, 360, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 360, 360, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 180, 180, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 180, 180, 64)      18496     
_________________________________________________________________
activation_6 (Activation)    (None, 180, 180, 64)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 90, 90, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 90, 90, 128)       73856     
__________

In [25]:
class BatchIndices(object):
    def __init__(self, n, bs, shuffle=False):
        self.n,self.bs,self.shuffle = n,bs,shuffle
        self.lock = threading.Lock()
        self.reset()

    def reset(self):
        self.idxs = (np.random.permutation(self.n) 
                     if self.shuffle else np.arange(0, self.n))
        self.curr = 0

    def __next__(self):
        with self.lock:
            if self.curr >= self.n: self.reset()
            ni = min(self.bs, self.n-self.curr)
            res = self.idxs[self.curr:self.curr+ni]
            self.curr += ni
            return res

In [26]:
class segm_generator(object):
    def __init__(self, x, y, bs=64, out_sz=(224,224), train=True):
        self.x, self.y, self.bs, self.train = x,y,bs,train
        self.n, self.ri, self.ci, _ = x.shape
        self.idx_gen = BatchIndices(self.n, bs, train)
        self.ro, self.co = out_sz
        self.ych = self.y.shape[-1] if len(y.shape)==4 else 1

    def get_slice(self, i,o):
        start = random.randint(0, i-o) if self.train else (i-o)
        return slice(start, start+o)

    def get_item(self, idx):
        condition =True
        while True:
            slice_r = self.get_slice(self.ri, self.ro)
            slice_c = self.get_slice(self.ci, self.co)
            x = self.x[idx, slice_r, slice_c]
            y = self.y[idx, slice_r, slice_c]
            if self.train and (random.random()>0.5): 
                y = y[:,::-1]
                x = x[:,::-1]
            # Modified to create patches with atleast one sea lion
            if np.sum(y)!=0:
                break
        return x, y

    def __next__(self):
        idxs = next(self.idx_gen)
        items = (self.get_item(idx) for idx in idxs)
        xs,ys = zip(*items)
        return np.stack(xs), np.stack(ys).reshape(len(ys), -1, self.ych)

In [8]:
def train():
    model= get_model()
    
    x_train,y_train= load_data('../experiment-charles/kernels/Kaggle-Sea-Lions-Solution-master/kaggle_data/train_images_512x512')
    train_generator = segm_generator(x_train, y_train, batch_size=batch_size, out_sz=(image,image_size) ,train=True)
    #test_generator = segm_generator(test_set, test_labels, 4, out_sz=lrg_sz, train=False)
        
    model.fit_generator(train_generator, steps_per_epoch=len(x_train) / batch_size, epochs=epochs)
    model.save(model_name+'_model.h5')
 
def create_submission():
    model = load_model(model_name+'_model.h5')
    
    n_test_images= 18636
    pred_arr= np.zeros((n_test_images,n_classes),np.int32)
    for k in range(0,n_test_images):
        image_path= 'kaggle_data/test_images_512x512/'+str(k)+'.png'
        print(image_path) #
        
        img= cv2.imread(image_path)
        img= img[None,...]
        pred= model.predict(img)
        pred= pred.astype(int)
        
        pred_arr[k,:]= pred
        
    print('pred_arr.shape', pred_arr.shape)
    pred_arr = pred_arr.clip(min=0)
    df_submission = pd.DataFrame()
    df_submission['test_id']= range(0,n_test_images)
    df_submission['adult_males']= pred_arr[:,0]
    df_submission['subadult_males']= pred_arr[:,1]
    df_submission['adult_females']= pred_arr[:,2]
    df_submission['juveniles']= pred_arr[:,3]
    df_submission['pups']= pred_arr[:,4]
    df_submission.to_csv(model_name+'_submission_moi.csv',index=False)
   
train()
create_submission()

TypeError: __init__() missing 2 required positional arguments: 'nb_row' and 'nb_col'